In [ ]:
# %pip install sdv

### 23.09.18 New Idea
- tps, latency를 모두 generate하지 않고 config만 generate한 후에 성능 prediction
- Optimization epoch 1000

### Data Prepare (Metric)

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

A_metrics = pd.read_csv("/home/sein/mk_config/ycsb_AA/results/external_metrics.csv")

In [2]:
A_metrics = pd.read_csv("/home/sein/mk_config/ycsb_AA/results/external_metrics.csv")
# AA_metrics = pd.read_csv("/home/sein/mk_config/ycsb_AA/results/external_metrics.csv")
# B_metrics = pd.read_csv("/home/sein/mk_config/ycsb_B/results/external_metrics.csv")

# metrics = pd.concat([A_metrics, AA_metrics, B_metrics], axis=0)
metrics = A_metrics.drop(['Unnamed: 0'], axis = 1)
# metrics.reset_index(inplace=True)
# metrics = metrics.drop(['index'], axis='columns')

In [3]:
metrics = metrics.replace([np.inf],9999999)

metrics


,tps,latency
0,0.216666,20847397
1,0.322221,12744942
2,3789.313102,1076
3,0.166667,12510551
4,0.266666,10686198
...,...,...
995,0.138889,11836756
996,0.149999,16386967
997,0.227778,19293708
998,0.188888,10529195


### Data Prepare (Config)

In [4]:
import random
import glob
import os
import sys
import pandas as pd

knob_list = glob.glob("/home/sein/mk_config/ycsb_AA/configs/my_*.cnf")


In [5]:
cnt = 0

for xx in range(len(knob_list)):
    path = "/home/sein/mk_config/ycsb_AA/configs/my_{}.cnf".format(xx)
    # knob_list = glob.glob("/home/sein/2023_EDBT/KCC_tpcc_dataset/my_*.cnf")
    a_all = pd.read_csv(path, sep="=", names=['Sample', 'value'], header=2)
    a_all = a_all.set_index("Sample")
    cur_all_df = a_all.T
    
    if cnt == 0:
        A_config = cur_all_df
    else :
        A_config = pd.concat([A_config, cur_all_df], axis=0)
    cnt += 1
A_config = A_config.reset_index()
A_config = A_config.drop(["index"],axis=1)
A_config = A_config.drop(A_config.columns[[0,1]], axis=1)


A_config

Sample,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,18000,4276224,945000,610000,1,4096000,432013312,0,29,...,159744,520000,91200,8000,63,7680,163577856,57344,104448,1
996,1,57000,5636096,245000,211000,1,8716288,268435456,1,24,...,455936,940000,89600,2000,29,11520,230686720,69632,61440,0
997,1,2000,622592,45000,537000,1,393216,671088640,7,20,...,308480,810000,66000,8000,48,12544,536870912,81920,108544,1
998,0,38000,5505024,700000,598000,0,9125888,834666496,3,17,...,404992,260000,70400,2000,57,3840,218103808,36864,63488,1


In [6]:
samples = pd.concat([A_config,metrics], axis=1)

samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit,tps,latency
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,13200,4000,18,11520,255852544,28672,94208,0,0.216666,20847397
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,91600,4000,17,15360,402653184,110592,71680,1,0.322221,12744942
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,15600,1,9,1024,851443712,8192,122880,0,3789.313102,1076
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,11200,10000,9,13568,163577856,16384,83968,0,0.166667,12510551
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,11600,4000,25,1536,348127232,126976,57344,0,0.266666,10686198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,18000,4276224,945000,610000,1,4096000,432013312,0,29,...,91200,8000,63,7680,163577856,57344,104448,1,0.138889,11836756
996,1,57000,5636096,245000,211000,1,8716288,268435456,1,24,...,89600,2000,29,11520,230686720,69632,61440,0,0.149999,16386967
997,1,2000,622592,45000,537000,1,393216,671088640,7,20,...,66000,8000,48,12544,536870912,81920,108544,1,0.227778,19293708
998,0,38000,5505024,700000,598000,0,9125888,834666496,3,17,...,70400,2000,57,3840,218103808,36864,63488,1,0.188888,10529195


In [7]:
### boolean에 해당하는 열 정리

discrete_columns = [samples.columns[0], samples.columns[5],samples.columns[10],samples.columns[13],
                    samples.columns[17],samples.columns[24],samples.columns[30],samples.columns[31],
                    samples.columns[32],samples.columns[34],samples.columns[36],samples.columns[37],
                    samples.columns[58],samples.columns[60],samples.columns[64],samples.columns[68],
                    samples.columns[72],samples.columns[73],samples.columns[74],samples.columns[75],
                    samples.columns[77],samples.columns[80],samples.columns[82],samples.columns[83],
                    samples.columns[90],samples.columns[91],samples.columns[92],samples.columns[93],
                    samples.columns[118],samples.columns[123],samples.columns[124],samples.columns[125],
                    samples.columns[126]]

In [8]:
### continuous 열 정리
all_columns = samples.columns
continuous_columns = all_columns.drop(discrete_columns)

In [9]:
for i in range(len(discrete_columns)):
    a = discrete_columns[i]
    samples = samples.astype({a:'int'})

In [10]:
for i in range(len(continuous_columns)):
    a = continuous_columns[i]
    samples = samples.astype({a:'float'})

### config - metric prediction (with raw data)

In [11]:
### TabNet
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import torch
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

X_all = np.array(A_config)
Y_all = np.array(metrics)

X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.3, shuffle=True)
X_scaler = MinMaxScaler().fit(X_train)
Y_scaler = MinMaxScaler().fit(y_train)

scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)
scaled_y_train = Y_scaler.transform(y_train)
scaled_y_test = Y_scaler.transform(y_test)

# Tabnet 모델 생성
regressor = TabNetRegressor(verbose = 10,seed = 42,optimizer_fn=torch.optim.AdamW) ### Basic

# 모델 학습
regressor.fit(X_train=scaled_X_train, y_train=scaled_y_train,
              eval_set=[(scaled_X_test, scaled_y_test)],
              patience=50, 
              batch_size = 64,
              max_epochs=10000,
              eval_metric=['mse'])

predictions = regressor.predict(scaled_X_test)

test_score = mean_squared_error(y_pred = predictions, y_true = scaled_y_test)
# 성능 평가
print('BEST VALID SCORE : ', regressor.best_cost)
# print('MSE_SCORE : ', test_score)
print('R2 SCORE : ' , r2_score(predictions, scaled_y_test))


epoch 0  | loss: 0.80765 | val_0_mse: 0.14754 |  0:00:00s
epoch 10 | loss: 0.05001 | val_0_mse: 0.06178 |  0:00:02s
epoch 20 | loss: 0.04528 | val_0_mse: 0.06087 |  0:00:03s
epoch 30 | loss: 0.03765 | val_0_mse: 0.06723 |  0:00:05s
epoch 40 | loss: 0.01745 | val_0_mse: 0.01482 |  0:00:07s
epoch 50 | loss: 0.01588 | val_0_mse: 0.01482 |  0:00:08s
epoch 60 | loss: 0.01185 | val_0_mse: 0.01009 |  0:00:10s
epoch 70 | loss: 0.01502 | val_0_mse: 0.01181 |  0:00:12s
epoch 80 | loss: 0.01104 | val_0_mse: 0.01224 |  0:00:13s
epoch 90 | loss: 0.01109 | val_0_mse: 0.01504 |  0:00:15s
epoch 100| loss: 0.01221 | val_0_mse: 0.01455 |  0:00:17s
epoch 110| loss: 0.01582 | val_0_mse: 0.01171 |  0:00:18s
epoch 120| loss: 0.01212 | val_0_mse: 0.01055 |  0:00:20s

Early stopping occurred at epoch 127 with best_epoch = 77 and best_val_0_mse = 0.00839
BEST VALID SCORE :  0.008391309574085435
R2 SCORE :  0.8174922626735889


In [39]:
np.set_printoptions(threshold=5)

print(predictions)
print('-------------------------------')
print(scaled_y_test)

[[ 0.00761061  0.55664396]
 [ 0.00649991  0.5448168 ]
 [ 0.0013749   0.54706484]
 ...
 [ 0.8731402  -0.00378415]
 [ 0.00303609  0.57134974]
 [ 0.00661602  0.49663788]]
-------------------------------
[[2.26870519e-05 5.06634955e-01]
 [2.13526510e-05 7.03470952e-01]
 [2.40214800e-05 6.94651441e-01]
 ...
 [8.99309013e-01 1.36295343e-05]
 [1.86834012e-05 5.08131264e-01]
 [1.60145184e-05 5.25723430e-01]]


### LHS SAMPLING

In [17]:
import pandas as pd

knob_info = pd.read_csv('Knob_Information_MySQL_v5.7.csv')

knob_min = knob_info['raw_min']
knob_max = knob_info['raw_max']


In [18]:
import numpy as np

def lhs_sampling(min_val, max_val, num_samples):
    # LHS 샘플 생성
    lhs_samples = np.linspace(min_val, max_val, num_samples, endpoint=False)
    np.random.shuffle(lhs_samples)
    return lhs_samples
samples = lhs_sampling(knob_min, knob_max, 2000)

# 생성된 LHS 샘플 출력
print(samples)
# samples.shape

[[1.18500000e-01 7.11088150e+03 1.24617318e+06 ... 1.64346880e+04
  1.64346880e+04 1.18500000e-01]
 [6.75500000e-01 4.05303245e+04 7.08446003e+06 ... 8.88714240e+04
  8.88714240e+04 6.75500000e-01]
 [6.12500000e-01 3.67503875e+04 6.42411520e+06 ... 8.06784000e+04
  8.06784000e+04 6.12500000e-01]
 ...
 [2.50000000e-01 1.50007500e+04 2.62451200e+06 ... 3.35360000e+04
  3.35360000e+04 2.50000000e-01]
 [4.28000000e-01 2.56805720e+04 4.49024819e+06 ... 5.66845440e+04
  5.66845440e+04 4.28000000e-01]
 [6.51000000e-01 3.90603490e+04 6.82765926e+06 ... 8.56852480e+04
  8.56852480e+04 6.51000000e-01]]


In [19]:
samples.shape

(2000, 138)

In [21]:
### LHS sampling으로 생성한 X값에 대한 scaling

from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

new_X = np.array(samples)
Z_scaler = MinMaxScaler().fit(new_X)

predictions_new = regressor.predict(new_X)


In [22]:
predictions_new.shape

(2000, 2)

### 실제로 확인

In [ ]:
# knob 실제로 벤치마킹할 때 predict한 metric값이 나오는지 확인

# all_test[500]
# all_test_pd = pd.DataFrame(all_test)
inverse_transformed_ex = Z_scaler.inverse_transform(all_test) 
# print(inverse_transformed_ex)
ex_A = inverse_transformed_ex[15] #500번째 테스트 샘플

inverse_transformed_ex_m = Y_scaler.inverse_transform(predictions) 
ex_A_met = inverse_transformed_ex_m[15] #500번째 성능

# ex_A , ex_A_met
# ex_A



In [ ]:
ex_A_p = pd.DataFrame(ex_A)
ex_A_met_p = pd.DataFrame(ex_A_met)

pd.options.display.float_format = '{:.5f}'.format
# real_ex_A = [math.exp(x) for x in ex_A_p]  # 각 값에 대한 지수 함수 값을 계산하여 실수로 변환

print(ex_A_p)
print(ex_A_met_p)

In [ ]:
ex_A_p[0][0]

In [ ]:
for i in range (len(A_config)):
    print('{} = {}'.format(A_config.columns[i],round(ex_A_p[0][i])))

In [ ]:
# a = Z_scaler.data_max_
# b = Z_scaler.data_min_
# (new_X[0]- b) / (a - b)

### metric 확인

In [ ]:
# metrics
# metrics.shape 1000,2
# new_pred = predictions[200:701] #scaling 되어있음

# inverse_new_pred = Y_scaler.inverse_transform(new_pred) 
# # print(inverse_new_pred)
# inverse_new_pred_pd = pd.DataFrame(inverse_new_pred)
# inverse_new_pred_pd.rename(columns={0: "tps", 1:"latency"}, inplace=True)
# # inverse_new_pred_pd

# new_metrics_re = pd.concat([metrics,inverse_new_pred_pd], axis=0)

# new_metrics_re = new_metrics_re.reset_index()
# new_metrics_re = new_metrics_re.drop(['index'], axis=1)
# new_metrics_re

In [23]:
### new_metrics_re = 원래 metric + predic_metric

predictions_new_df = pd.DataFrame(predictions_new)
inverse_new_pred = Y_scaler.inverse_transform(predictions_new) 
# print(inverse_new_pred)
inverse_new_pred_pd = pd.DataFrame(inverse_new_pred)
inverse_new_pred_pd.rename(columns={0: "tps", 1:"latency"}, inplace=True)
# inverse_new_pred_pd

new_metrics_re = pd.concat([metrics,inverse_new_pred_pd], axis=0)
# new_metrics_re = new_metrics_re.drop(['index'], axis=1)

new_metrics_re = new_metrics_re.reset_index()


In [24]:
new_metrics_re = new_metrics_re.drop(['index'], axis=1)

new_metrics_re


,tps,latency
0,2.166658e-01,2.084740e+07
1,3.222207e-01,1.274494e+07
2,3.789313e+03,1.076000e+03
3,1.666666e-01,1.251055e+07
4,2.666662e-01,1.068620e+07
...,...,...
2995,4.595821e+11,6.415443e+13
2996,5.339939e+09,7.789562e+13
2997,7.157562e+09,1.044090e+14
2998,1.225263e+10,1.787300e+14


### 인자 정리
- new_X = samples = LHS Sampling으로 생성한 데이터
<!-- - new_metrics = 생성한 데이터에 대해서 TabNET이 예측한 metrics + 원래 metrics (scaling X) -->
- new_metrics_re = 원래 metric (A_metrics) + 생성한 데이터로 예측한 metric (scaling X)
- new_Samples = 원래 config + 생성한 config (scaling X)
- newnewnew = AutoEncoder에 넣을 수 있는 데이터셋

In [25]:
# inversed_new_config = pd.DataFrame(inverse_transformed_samples)
new_X_pd = pd.DataFrame(new_X)
new_X_pd = round(new_X_pd).astype(int)
for i in range(len(new_X_pd.columns)):
    new_X_pd.rename(columns={new_X_pd.columns[i]: A_config.columns[i]}, inplace=True)   
    
new_Samples = pd.concat([A_config,new_X_pd] , axis=0)

In [26]:
new_Samples = new_Samples.reset_index()
new_Samples = new_Samples.drop(['index'], axis=1)
new_Samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1,59790,10449074,996500,996500,1,10449074,1069983728,7,30,...,522453,996500,99651,9965,64,16327,1069983731,130617,130617,1
2996,0,11191,1958926,186500,186500,0,1958926,200252850,1,6,...,97793,186500,18975,1866,13,3056,200253683,25278,25278,0
2997,0,15001,2624512,250000,250000,0,2624512,268435456,2,8,...,131084,250000,25300,2501,17,4096,268436224,33536,33536,0
2998,0,25681,4490248,428000,428000,0,4490248,459561501,3,13,...,224404,428000,43029,4281,28,7012,459562086,56685,56685,0


In [20]:
new_Samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,13411,2346748,223500,223500,0,2346748,239981298,2,7,...,117191,223500,22661,2236,15,3662,239982093,30090,30090,0
2996,0,9061,1586827,151000,151000,0,1586827,162135015,1,5,...,79181,151000,15440,1511,11,2474,162135885,20661,20661,0
2997,0,22921,4008092,382000,382000,0,4008092,410169377,3,11,...,200288,382000,38447,3821,25,6259,410170010,50702,50702,0
2998,1,34500,6031053,575000,575000,1,6031053,617401549,4,17,...,301472,575000,57670,5750,37,9421,617401984,75802,75802,1


In [24]:
new_metrics_re


,tps,latency
0,2.166658e-01,2.084740e+07
1,3.222207e-01,1.274494e+07
2,3.789313e+03,1.076000e+03
3,1.666666e-01,1.251055e+07
4,2.666662e-01,1.068620e+07
...,...,...
2995,7.569932e+07,-1.416420e+10
2996,5.200018e+07,-9.755034e+09
2997,1.275102e+08,-2.380289e+10
2998,1.905981e+08,-3.553872e+10


### NEW_IDEA
### AE Train set에 추가 (1000 + 10000) AUG == new_Samples

In [27]:
for i in range(len(new_X_pd.columns)):
    new_X_pd.rename(columns={new_X_pd.columns[i]: A_config.columns[i]}, inplace=True)   
    
new_Samples = pd.concat([A_config,new_X_pd] , axis=0)

new_Samples = new_Samples.reset_index()
new_Samples = new_Samples.drop(['index'], axis=1)
# new_Samples

newnewwnew = pd.concat([new_Samples, new_metrics_re], axis = 1)

In [26]:
new_metrics_re

,tps,latency
0,2.166658e-01,2.084740e+07
1,3.222207e-01,1.274494e+07
2,3.789313e+03,1.076000e+03
3,1.666666e-01,1.251055e+07
4,2.666662e-01,1.068620e+07
...,...,...
2995,7.569932e+07,-1.416420e+10
2996,5.200018e+07,-9.755034e+09
2997,1.275102e+08,-2.380289e+10
2998,1.905981e+08,-3.553872e+10


In [27]:
new_Samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,13411,2346748,223500,223500,0,2346748,239981298,2,7,...,117191,223500,22661,2236,15,3662,239982093,30090,30090,0
2996,0,9061,1586827,151000,151000,0,1586827,162135015,1,5,...,79181,151000,15440,1511,11,2474,162135885,20661,20661,0
2997,0,22921,4008092,382000,382000,0,4008092,410169377,3,11,...,200288,382000,38447,3821,25,6259,410170010,50702,50702,0
2998,1,34500,6031053,575000,575000,1,6031053,617401549,4,17,...,301472,575000,57670,5750,37,9421,617401984,75802,75802,1


In [28]:
newnewwnew

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit,tps,latency
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,13200,4000,18,11520,255852544,28672,94208,0,2.166658e-01,2.084740e+07
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,91600,4000,17,15360,402653184,110592,71680,1,3.222207e-01,1.274494e+07
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,15600,1,9,1024,851443712,8192,122880,0,3.789313e+03,1.076000e+03
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,11200,10000,9,13568,163577856,16384,83968,0,1.666666e-01,1.251055e+07
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,11600,4000,25,1536,348127232,126976,57344,0,2.666662e-01,1.068620e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1,59790,10449074,996500,996500,1,10449074,1069983728,7,30,...,99651,9965,64,16327,1069983731,130617,130617,1,4.595821e+11,6.415443e+13
2996,0,11191,1958926,186500,186500,0,1958926,200252850,1,6,...,18975,1866,13,3056,200253683,25278,25278,0,5.339939e+09,7.789562e+13
2997,0,15001,2624512,250000,250000,0,2624512,268435456,2,8,...,25300,2501,17,4096,268436224,33536,33536,0,7.157562e+09,1.044090e+14
2998,0,25681,4490248,428000,428000,0,4490248,459561501,3,13,...,43029,4281,28,7012,459562086,56685,56685,0,1.225263e+10,1.787300e+14


### 원래 sample + 생성한 sample에 대해 AE가 되는지 확인

In [29]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import  TensorDataset, DataLoader

X_train, X_test = train_test_split(newnewwnew, test_size=0.2, shuffle=True)

scaler = MinMaxScaler()
# new_train = pd.concat([X_train,full_samples])
X_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)


dataset_tr = TensorDataset(torch.tensor(X_scaler))
dataset_te = TensorDataset(torch.tensor(X_test_scaler))

trainloader = torch.utils.data.DataLoader(dataset_tr, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset_te, batch_size=128, shuffle=True)


In [30]:
### 원본

import torch.nn as nn

class Autoencoder(nn.Module):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = nn.Sequential(
        # nn.Linear(140,64),
        # nn.ReLU(),
        # nn.Dropout(p=0.3),
        # nn.Linear(64,32),
        # nn.ReLU(),
        # nn.Linear(32,16), #잠재변수 15개로 줄임 
        # nn.Sigmoid())
        nn.Linear(140,128),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32,16), #잠재변수 15개로 줄임 
        #잠재변수 15개로 줄임 
        nn.Sigmoid())
    
    self.decoder = nn.Sequential(
        # nn.Linear(16,32),
        # nn.ReLU(),
        # nn.Linear(32,64),
        # nn.ReLU(),
        # nn.Linear(64, 140), 
        # nn.Sigmoid()
        nn.Linear(16,32),
        nn.ReLU(),
        nn.Linear(32,64),
        nn.ReLU(),
        # nn.Dropout(p=0.2),
        nn.Linear(64,128),
        nn.ReLU(),
        nn.Linear(128, 140), 
        nn.Sigmoid()
    )
  
  #인코더와 디코더 연산을 차례대로 수행하도록 설정 
  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [56]:
# from torch import device
from torch import optim
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model =Autoencoder().to(device)
model = Autoencoder().to(device)
# model = Autoencoder()


critertion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#lr=1e-4
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trainloss = []
validationloss = []
epoch_list = []

for epoch in range(4000):
  running_loss = 0
  model.train()
  
  for data in trainloader:
    inputs = data[0].float().to(device)
    optimizer.zero_grad()
    outputs = model(inputs) 
    loss = critertion(inputs, outputs)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    
  train_loss = running_loss / len(trainloader)
  trainloss.append(train_loss)
  # epoch_list.append(epoch)
  
  if epoch % 300 == 0:
    total_val_loss = 0
    model.eval()
    with torch.no_grad():
      running_loss = 0
      
      for data in testloader:
        inputs = data[0].float().to(device)
        # optimizer.zero_grad()
        outputs = model(inputs) 
        loss = critertion(inputs, outputs)
        running_loss += loss.item()
        
      total_val_loss = running_loss / len(testloader)
      validationloss.append(total_val_loss)
        
    print('[%d] tr_loss : %.3f | val_loss : %.3f' %(epoch +1, train_loss, total_val_loss))

[1] tr_loss : 0.127 | val_loss : 0.128
[301] tr_loss : 0.040 | val_loss : 0.049
[601] tr_loss : 0.037 | val_loss : 0.050
[901] tr_loss : 0.034 | val_loss : 0.049
[1201] tr_loss : 0.032 | val_loss : 0.049
[1501] tr_loss : 0.031 | val_loss : 0.051
[1801] tr_loss : 0.030 | val_loss : 0.050
[2101] tr_loss : 0.029 | val_loss : 0.050
[2401] tr_loss : 0.029 | val_loss : 0.051
[2701] tr_loss : 0.028 | val_loss : 0.052
[3001] tr_loss : 0.028 | val_loss : 0.051
[3301] tr_loss : 0.027 | val_loss : 0.053
[3601] tr_loss : 0.027 | val_loss : 0.052
[3901] tr_loss : 0.026 | val_loss : 0.051


In [ ]:
inputs.shape , outputs.shape

### Optimization in latent space
- Prediction

In [50]:
import torch
import torch.nn as nn
import numpy as np

for data in testloader:
    inputs = data[0].float().to(device)
    with torch.no_grad():
        encoded_vector = model.encoder(inputs)
        print("Encoded Latent Vector:", encoded_vector)


Encoded Latent Vector: tensor([[9.9590e-01, 9.9994e-01, 9.9997e-01,  ..., 1.0343e-08, 1.8569e-03,
         8.4013e-01],
        [9.9406e-01, 3.4512e-01, 3.2576e-02,  ..., 1.3790e-02, 8.1211e-01,
         3.4959e-01],
        [9.9936e-01, 5.6140e-01, 5.4231e-04,  ..., 4.1705e-04, 3.1179e-01,
         5.3268e-02],
        ...,
        [5.5635e-01, 3.6206e-01, 2.0626e-01,  ..., 3.8067e-01, 5.5052e-01,
         2.5642e-01],
        [9.9282e-01, 2.9986e-01, 1.3604e-02,  ..., 1.5869e-03, 8.6443e-01,
         2.2016e-01],
        [3.0649e-01, 3.5248e-01, 7.1561e-01,  ..., 6.2281e-01, 5.2784e-01,
         4.6933e-02]], device='cuda:0')
Encoded Latent Vector: tensor([[4.4642e-01, 9.9791e-01, 9.9071e-01,  ..., 4.9650e-03, 6.4446e-01,
         2.6101e-02],
        [6.7554e-01, 2.8531e-01, 3.2694e-01,  ..., 9.6129e-01, 4.5558e-01,
         7.1906e-02],
        [1.6677e-01, 2.5359e-01, 6.1509e-02,  ..., 6.4537e-01, 6.5676e-01,
         2.4141e-01],
        ...,
        [9.9973e-01, 9.9988e-01, 1.00

In [52]:
print(inputs.shape)
print(encoded_vector.shape)

torch.Size([88, 140])
torch.Size([88, 32])


In [ ]:
latent_space = encoded_vector
print(latent_space.shape)

### 다시 확인

In [29]:
### test에 사용한 real 데이터의 metric값


for data in testloader:
    inputs = data[0].float().to(device)
    inputs_pd = pd.DataFrame(inputs.cpu().numpy())
    new_metric = inputs_pd.iloc[:,[138,139]]
    # optimizer.zero_grad()
    # print(new_metric)

In [30]:
new_metric

,138,139
0,2.582464e-04,3.708011e-06
1,2.115008e-04,1.103781e-05
2,1.433071e-04,2.173069e-05
3,2.805194e-04,2.155779e-07
4,2.787602e-13,4.390574e-05
5,2.818942e-04,0.000000e+00
6,2.252495e-04,8.881979e-06
7,2.018767e-04,1.254688e-05
8,2.813443e-04,8.622207e-08
9,4.181365e-13,4.390625e-05


In [ ]:
new_metric.shape

In [ ]:
inputs.shape

In [31]:
new_metric

,138,139
0,2.582464e-04,3.708011e-06
1,2.115008e-04,1.103781e-05
2,1.433071e-04,2.173069e-05
3,2.805194e-04,2.155779e-07
4,2.787602e-13,4.390574e-05
5,2.818942e-04,0.000000e+00
6,2.252495e-04,8.881979e-06
7,2.018767e-04,1.254688e-05
8,2.813443e-04,8.622207e-08
9,4.181365e-13,4.390625e-05


### TabNet (Optimization과정에서)

In [ ]:
samples.shape

In [ ]:
newnewwnew

In [57]:
### 원래 데이터 + LHS Sampling data 차원을 32로 줄인 latent space 구하기
### X = latent config (# 1500) , Y = metrics (# 2)

import torch
import pandas as pd
import numpy as np

scaler_x = MinMaxScaler()
samples_scaler = scaler_x.fit_transform(newnewwnew)

# samples_np = samples_scaler.values
samples_torch = torch.Tensor(samples_scaler).to(device)

with torch.no_grad():
    encoded_vector_BO = model.encoder(samples_torch)
    print("BO'S Encoded Latent Vector:", encoded_vector_BO)

BO'S Encoded Latent Vector: tensor([[0.1761, 0.5794, 0.0221,  ..., 0.0506, 0.7647, 0.4478],
        [0.8110, 0.4584, 0.8484,  ..., 0.6502, 0.7807, 0.1270],
        [0.6617, 0.6022, 0.5477,  ..., 0.3435, 0.7851, 0.5362],
        ...,
        [0.0691, 0.6007, 0.0534,  ..., 0.2040, 0.1943, 0.4384],
        [0.8624, 0.5552, 0.9065,  ..., 0.8706, 0.0610, 0.1407],
        [0.0118, 0.7625, 0.0018,  ..., 0.0658, 0.0921, 0.3916]],
       device='cuda:0')


In [58]:
encoded_vector_BO.shape

torch.Size([3000, 16])

In [ ]:
new_Samples

In [ ]:
# encoded_vector_BO.shape
new_metrics_re

In [60]:
### TabNet
### X = encoded_vector_BO (Scaling O) , Y = metrics (Scaling X)

from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X_latent = np.array(encoded_vector_BO.cpu().numpy())
Y_latent = np.array(new_metrics_re)

lt_X_train, lt_X_test, lt_y_train, lt_y_test = train_test_split(X_latent,Y_latent,test_size=0.3, shuffle=True)
Y_scaler = MinMaxScaler().fit(lt_y_train)

scaled_lt_y_train = Y_scaler.transform(lt_y_train)
scaled_lt_y_test = Y_scaler.transform(lt_y_test)

# Tabnet 모델 생성
lt_regressor = TabNetRegressor(verbose = 10,seed = 42,optimizer_fn=torch.optim.AdamW) 

# 모델 학습
lt_regressor.fit(X_train=lt_X_train, y_train=scaled_lt_y_train,
              eval_set=[(lt_X_test, scaled_lt_y_test)],
              patience=50, 
              batch_size = 128,
              max_epochs=1000,
              eval_metric=['mse'])

# 테스트 데이터로 예측
lt_predictions = lt_regressor.predict(lt_X_test)

# 성능 평가
print('BEST VALID SCORE : ', lt_regressor.best_cost)
print('R2 SCORE : ' , r2_score(scaled_lt_y_test, lt_predictions))

# print('MSE_SCORE : ', test_score)


epoch 0  | loss: 0.70264 | val_0_mse: 0.11127 |  0:00:00s
epoch 10 | loss: 0.00231 | val_0_mse: 0.13408 |  0:00:03s
epoch 20 | loss: 0.0009  | val_0_mse: 0.12431 |  0:00:05s
epoch 30 | loss: 0.00103 | val_0_mse: 0.00179 |  0:00:08s
epoch 40 | loss: 0.00096 | val_0_mse: 0.00037 |  0:00:11s
epoch 50 | loss: 0.00146 | val_0_mse: 0.00041 |  0:00:14s
epoch 60 | loss: 0.00089 | val_0_mse: 0.00107 |  0:00:16s
epoch 70 | loss: 0.00226 | val_0_mse: 0.00218 |  0:00:19s
epoch 80 | loss: 0.0005  | val_0_mse: 0.0005  |  0:00:22s
epoch 90 | loss: 0.00058 | val_0_mse: 0.00023 |  0:00:25s
epoch 100| loss: 0.0004  | val_0_mse: 0.00048 |  0:00:27s
epoch 110| loss: 0.00087 | val_0_mse: 0.00183 |  0:00:30s
epoch 120| loss: 0.00046 | val_0_mse: 0.00052 |  0:00:33s
epoch 130| loss: 0.00247 | val_0_mse: 0.00949 |  0:00:36s
epoch 140| loss: 0.00057 | val_0_mse: 0.00089 |  0:00:38s
epoch 150| loss: 0.00101 | val_0_mse: 0.0005  |  0:00:41s
epoch 160| loss: 0.00125 | val_0_mse: 0.0055  |  0:00:44s

Early stoppin

In [63]:
print(lt_predictions)
print('-----------------------------------------------')
print(scaled_lt_y_test)

[[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 ...
 [ 0.00071219 -0.00268302]
 [ 0.00072886 -0.00271483]
 [ 0.          0.        ]]
-----------------------------------------------
[[6.59663204e-13 8.52900483e-06]
 [4.10457294e-13 8.52905660e-06]
 [6.36125876e-09 8.52678553e-06]
 ...
 [1.45836742e-04 6.46241822e-06]
 [1.03140924e-04 7.06210675e-06]
 [3.81139128e-13 8.52904590e-06]]


In [64]:
print(lt_X_test)
print('-----------------------------------------------')
print(scaled_lt_y_test)

[[5.23794711e-01 4.41426307e-01 9.18241203e-01 ... 2.50850737e-01
  9.71134543e-01 2.23279044e-01]
 [1.46030575e-01 8.41418207e-01 8.62705827e-01 ... 2.99709439e-01
  2.15932012e-01 9.25122380e-01]
 [6.96771324e-01 3.86776835e-01 6.51102424e-01 ... 4.75303620e-01
  5.61440706e-01 2.82645792e-01]
 ...
 [8.99117906e-03 7.92759180e-01 1.78370182e-03 ... 6.31797314e-02
  1.09356776e-01 4.46293890e-01]
 [3.10261291e-03 8.73375118e-01 1.50288703e-04 ... 2.23087948e-02
  1.10358626e-01 3.98751765e-01]
 [6.00627899e-01 3.80117774e-01 4.84747857e-01 ... 2.17551485e-01
  5.63026488e-01 1.58527270e-01]]
-----------------------------------------------
[[6.59663204e-13 8.52900483e-06]
 [4.10457294e-13 8.52905660e-06]
 [6.36125876e-09 8.52678553e-06]
 ...
 [1.45836742e-04 6.46241822e-06]
 [1.03140924e-04 7.06210675e-06]
 [3.81139128e-13 8.52904590e-06]]


In [49]:
scaled_lt_y_test, lt_predictions

(array([[2.78525694e-13, 8.53510497e-06],
        [4.37804669e-04, 2.36799726e-06],
        [8.77058296e-01, 8.77059645e-01],
        ...,
        [5.86363873e-14, 8.53520716e-06],
        [9.25035584e-01, 9.25036309e-01],
        [3.07845338e-13, 8.53618058e-06]]),
 array([[ 0.00265984,  0.00440948],
        [ 0.02361161,  0.01335548],
        [ 0.6752999 ,  0.6727661 ],
        ...,
        [ 0.00570246,  0.00457435],
        [ 0.680132  ,  0.6671624 ],
        [-0.01929884, -0.01031821]], dtype=float32))

In [44]:
lt_predictions , scaled_lt_y_test

(array([[ 8.6209309e-01,  8.6343294e-01],
        [-1.7992589e-03, -1.6201108e-03],
        [-4.8071379e-05, -4.7473618e-04],
        ...,
        [-6.2339089e-04, -7.5627741e-04],
        [ 4.8741698e-04, -1.4071446e-04],
        [ 8.0739552e-01,  8.0812407e-01]], dtype=float32),
 array([[8.81996862e-01, 8.81998445e-01],
        [2.03473552e-05, 8.23526950e-06],
        [1.00603566e-04, 7.10807332e-06],
        ...,
        [3.22665924e-13, 8.53940563e-06],
        [1.78271072e-04, 6.01720332e-06],
        [7.97495617e-01, 7.97496896e-01]]))

In [ ]:
###Gradient Boosting Regressor

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, precision_score
import numpy as np

X_latent = np.array(encoded_vector_BO.cpu().numpy())
Y_latent = np.array(metrics)

lt_X_train, lt_X_test, lt_y_train, lt_y_test = train_test_split(X_latent,Y_latent,test_size=0.2, shuffle=True)
# X_scaler = MinMaxScaler().fit(lt_X_train)
Y_scaler = MinMaxScaler().fit(lt_y_train)

# scaled_lt_X_train = X_scaler.transform(lt_X_train)
# scaled_lt_X_test = X_scaler.transform(lt_X_test)
scaled_lt_y_train = Y_scaler.transform(lt_y_train)
scaled_lt_y_test = Y_scaler.transform(lt_y_test)

# xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
#                            colsample_bytree=1, max_depth=7)

estimator = GradientBoostingRegressor(
                loss = 'squared_error',
                criterion= 'squared_error',
                n_estimators= 5000,
                min_samples_leaf= 20,
                max_depth= 1,
                learning_rate= 0.001,
                random_state=0
                
)

my_model_pred = MultiOutputRegressor(estimator = estimator).fit(lt_X_train, scaled_lt_y_train)


pred = my_model_pred.predict(lt_X_test)

#rmse
# print('정확도 : ', pred.score(scaled_lt_y_test, pred))
print('R2_SCORE : ', r2_score(scaled_lt_y_test, pred))
# print('PCC : ', precision_score(y_test, pred, average='weighted'))

# rmse r2 pcc




In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score

# Gradient Boosting Regressor 모델 설정
estimator = GradientBoostingRegressor()

# MultiOutputRegressor로 감싸서 다중 출력 회귀 모델 생성
mod = MultiOutputRegressor(estimator)

# GridSearchCV를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'estimator__max_depth' : [3,2,1],
    'estimator__min_samples_leaf' : [12,20],
}


# GridSearchCV 객체 생성
grid_search = GridSearchCV(mod, param_grid, cv=5, scoring=make_scorer(r2_score), n_jobs=-1)

# 모델 학습 및 최적 하이퍼파라미터 탐색
grid_search.fit(lt_X_train, scaled_lt_y_train)

# 최적 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)
print("Best R2 Score:", grid_search.best_score_)


In [ ]:
lt_X_test.shape, lt_y_test.shape

In [ ]:
lt_X_train

In [ ]:
scaled_lt_y_train

In [ ]:
# predictions, scaled_y_test
from sklearn.metrics import r2_score

# r2_score(lt_predictions, lt_y_test)
r2_score(scaled_lt_y_test, lt_predictions)



In [35]:
latent_pd = pd.DataFrame(encoded_vector_BO.cpu().numpy())
latent_pd_T = latent_pd.T
latent_pd_T.index
# latent_pd_T

RangeIndex(start=0, stop=32, step=1)

In [ ]:
latent_pd_T

In [37]:
### BO 코드
from sklearn.preprocessing import MinMaxScaler
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

class BO(object):
    def __init__(self, iteration, configs, metrics, config_info_path=None):
        self.iteration = iteration
        self.configs = configs
        self.metrics = metrics
        self.config_info_path = config_info_path
        # self.min_max_same_knobs = []
        
        self._get_config_info()
        self._init_pbounds()
    
    def _get_config_info(self):
        if self.config_info_path is None:
            self.config_info = pd.read_csv('/home/sein/mk_config/Knob_Information_MySQL_v5.7.csv', index_col=0)
        else:
            self.config_info = pd.read_csv(self.config_info_path, index_col=0)
        
        # if self.top_z_knob is not None:
        #     self.config_info = self.config_info.loc[self.top_z_knob]
        #     self.configs = self.configs[self.top_z_knob]
            

#     def _get_history(self):
#         self.history_configs = self.smac.runhistory.get_configs()
    
    def _init_pbounds(self):
        self.pbounds = {}
        
        for v in latent_pd_T.index:
            # self.pbounds[v] = (0,18)
            self.pbounds[str(v)] = (0, 1)
            
    
    def train_regression_model(self):
        X_all = np.array(self.configs)
        Y_all = np.array(self.metrics)
        
        cnt = 0
        while(True):
            X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.3, shuffle=True)         
            
            self.X_scaler = MinMaxScaler().fit(X_train)
            self.y_scaler = MinMaxScaler().fit(y_train)

            # scaled_X_train = self.X_scaler.transform(X_train)
            # scaled_X_test = self.X_scaler.transform(X_test)
            scaled_y_train = self.y_scaler.transform(y_train)
            scaled_y_test = self.y_scaler.transform(y_test)
            
            
            estimator = TabNetRegressor(verbose = 100, seed = 42, optimizer_fn=torch.optim.AdamW)

            estimator.fit(X_train=X_train, y_train=scaled_y_train,
              eval_set=[(X_test, scaled_y_test)],
              patience=50, 
              batch_size = 128,
              max_epochs=10000,
              eval_metric=['mse'])
            pred = estimator.predict(X_test)
            accuracy = mean_squared_error(y_pred = pred, y_true = scaled_y_test)
            r2_score_ = r2_score(scaled_y_test, pred)

            cnt += 1
            print(cnt, accuracy)
            print(r2_score_)
            if r2_score_ > 0.90 or cnt > 10:
                break


            # print('BEST VALID SCORE : ', regressor.best_cost)

        self.model = estimator
    
    def _target_function(self, **kwargs):
        x = np.fromiter(kwargs.values(), dtype=float)        
        # scaled_X = self.X_scaler.transform([x])
        res = self.model.predict(x)
        res = res[:,0] / res[:,1]
        # for name in self.config_info.index:
        #     knob = self.config_info.loc[name]
        #     knob_min = knob['raw_min']
        #     knob_max = knob['raw_max']
            
            # if self.reduce_search_space:
            #     knob_min = self.best_configs[name].min()
            #     knob_max = self.best_configs[name].max()
            #     if knob_min == knob_max:
            #         self.min_max_same_knobs.append(name)
            #         knob_max += 1
            # with torch.no_grad():
            #     scaled_X_t = torch.tensor(scaled_X)
            #     scaled_X_t = scaled_X_t.to('cuda:0')
            #     if model.decoder(scaled_X_t) < knob_min:
            #         res == 0
            #     elif model.decoder(scaled_X_t) > knob_max:
            #         res == 0
        
        
        return res.squeeze()
        # return res
    
    
    def tune(self):
        self.optimizer = BayesianOptimization(f=self._target_function, pbounds=self.pbounds, verbose=2, random_state=2)
        # self.optimizer = BayesianOptimization(f=self._target_function, verbose=2, random_state=2)
        self.acquisition_function = UtilityFunction(kind="ei", xi=0.01)
        self.optimizer.maximize(n_iter=self.iteration, acquisition_function=self.acquisition_function)
    
    
    def plot_history(self):
        self.y_obs = - np.array([res["target"] for res in self.optimizer.res])
        
        self.his_inc = []
        inc = np.inf
        ## Get minimum results on each iteration
        for res in self.y_obs:
            if res < inc:
                inc = res
            self.his_inc.append(inc)
#             res.append(his_res)

        plt.plot(self.his_inc)
        plt.xlabel('Iteration')
        plt.ylabel('result')
        plt.show()

### 밑에 다 돌린거고 decoding해서 값 확인해보기

In [ ]:
tuner1 = BO(iteration=1000, 
           configs=encoded_vector_BO.cpu().numpy(),
           metrics=new_metrics_re
           )
tuner1.train_regression_model()
tuner1.tune()
# tuner1.plot_history()

In [47]:
ex_data = [0.2515,  0.3851, 0.1983, 0.2072, 0.04203, 0.6034, 0.7527, 0.309, 0.01957, 0.107,
           0.6571, 0.7406, 0.9737, 0.1054, 0.5471, 0.467, 0.3135, 0.9268, 0.1925, 0.1914, 0.7365,
           0.4656, 0.9693, 0.5216, 0.2464, 0.4485, 0.282, 0.4303, 0.4483, 0.5755, 0.1848, 0.99]

In [56]:
ex_data = [0.2168, 0.0, 0.138, 1.0, 0.3111, 0.333, 0.6079, 0.1281, 0.0, 0.2796, 0.3718, 0.3238, 0.5304,
           0.474, 0.1646, 0.5807, 0.9193, 0.09729, 0.1823, 0.1282, 0.3789, 0.4876, 0.9011, 0.1251, 0.3144,
           0.04871, 0.8719, 1.0, 0.5318, 0.5375, 0.7821, 0.865]

In [52]:
ex_data = [0.8619, 0.5249, 0.6002, 0.9991, 0.3543, 0.5662, 0.6583, 0.9311, 0.7842, 0.05524,
           0.02226, 0.5493, 0.7891, 0.3761, 0.1813, 0.631, 0.4767, 0.1582, 0.9703, 0.9681,
           0.9885, 0.0, 0.8136, 0.3381, 0.3356, 0.998, 0.8706, 0.2109, 0.389, 0.1532, 0.0, 0.9467  ]

In [58]:
ex_data = torch.tensor(ex_data)
ex_data = ex_data.to('cuda:0')  # ex_data를 GPU로 이동

with torch.no_grad():
    decode_value = model.decoder(ex_data)
    print("Decoded Value:", decode_value)

Decoded Value: tensor([6.4824e-01, 8.2373e-01, 5.4986e-01, 4.6874e-01, 5.0770e-01, 1.3226e-07,
        7.3256e-01, 4.1707e-01, 2.4584e-01, 5.5880e-01, 4.5944e-03, 3.6874e-01,
        4.4241e-01, 1.0000e+00, 4.1733e-01, 7.0821e-01, 4.6285e-01, 1.4779e-01,
        1.9396e-01, 3.4814e-01, 4.1392e-01, 3.0256e-01, 3.6378e-01, 7.0773e-01,
        8.7684e-01, 4.3775e-01, 4.7638e-01, 4.9350e-01, 5.1865e-01, 1.3259e-01,
        6.5164e-01, 4.1798e-01, 3.8149e-01, 6.0668e-01, 2.6327e-01, 6.7646e-01,
        2.9503e-05, 9.9994e-01, 5.3028e-01, 6.0091e-01, 6.0628e-01, 7.1926e-01,
        4.7253e-01, 5.9855e-01, 2.6013e-01, 4.4548e-01, 3.2324e-01, 5.8244e-01,
        8.3967e-01, 1.3947e-01, 5.7830e-01, 6.3821e-01, 4.8185e-01, 4.6733e-01,
        3.5292e-01, 6.0820e-01, 5.9133e-01, 4.2697e-01, 5.0241e-01, 1.8115e-01,
        3.1135e-01, 4.6330e-01, 3.3856e-01, 3.1810e-01, 9.9703e-01, 3.5718e-01,
        5.0331e-01, 2.6912e-01, 9.9668e-01, 2.4544e-01, 5.7020e-01, 4.3492e-01,
        9.9931e-01, 2.790

In [59]:
for x in decode_value:
    print(x)

tensor(0.6482, device='cuda:0')
tensor(0.8237, device='cuda:0')
tensor(0.5499, device='cuda:0')
tensor(0.4687, device='cuda:0')
tensor(0.5077, device='cuda:0')
tensor(1.3226e-07, device='cuda:0')
tensor(0.7326, device='cuda:0')
tensor(0.4171, device='cuda:0')
tensor(0.2458, device='cuda:0')
tensor(0.5588, device='cuda:0')
tensor(0.0046, device='cuda:0')
tensor(0.3687, device='cuda:0')
tensor(0.4424, device='cuda:0')
tensor(1., device='cuda:0')
tensor(0.4173, device='cuda:0')
tensor(0.7082, device='cuda:0')
tensor(0.4629, device='cuda:0')
tensor(0.1478, device='cuda:0')
tensor(0.1940, device='cuda:0')
tensor(0.3481, device='cuda:0')
tensor(0.4139, device='cuda:0')
tensor(0.3026, device='cuda:0')
tensor(0.3638, device='cuda:0')
tensor(0.7077, device='cuda:0')
tensor(0.8768, device='cuda:0')
tensor(0.4378, device='cuda:0')
tensor(0.4764, device='cuda:0')
tensor(0.4935, device='cuda:0')
tensor(0.5186, device='cuda:0')
tensor(0.1326, device='cuda:0')
tensor(0.6516, device='cuda:0')
tensor(0

In [60]:
import math


real_values = [math.exp(x) for x in decode_value]  #지수함수 없는 상태

print(real_values)

[1.912178995759835, 2.2789735949138907, 1.7330114134427155, 1.597984258355564, 1.6614631920616965, 1.000000132257975, 2.0803938716322485, 1.5175015913200782, 1.2786893807613118, 1.7485660432166548, 1.0046049419613148, 1.4459078578027134, 1.5564478061710612, 2.718281828459045, 1.5179080834726542, 2.030358978378201, 1.5885974568696961, 1.1592693894888686, 1.2140511525883404, 1.4164299047781657, 1.5127391014767344, 1.3533162940503463, 1.4387626148435155, 2.029378115761251, 2.403296714663861, 1.5492245906112134, 1.6102346297956847, 1.638033859426608, 1.6797570348813826, 1.1417782282184568, 1.9186761979531235, 1.5188868792509453, 1.4644623873146607, 1.8343245848565575, 1.301183794193486, 1.966903668002052, 1.000029503009967, 2.7181259675492755, 1.6994156630544832, 1.8237857419154884, 1.8335980932324465, 2.0529127117365844, 1.6040440646668888, 1.8194739103291697, 1.2971037858974417, 1.5612458147856194, 1.3815925051603606, 1.7903941420867773, 2.31559663458674, 1.1496588116366608, 1.7830083567

In [61]:
### 원본

import numpy as np

# real_values 리스트를 NumPy 배열로 변환
real_values = np.array(real_values)
rescaled_actual = scaler.inverse_transform(real_values.reshape(1,-1))

In [62]:
# rescaled_actual.shape
# real_values.shape
rescaled_actual


array([[1.91217900e+00, 1.36737137e+05, 1.81239031e+07, 1.58621046e+06,
        1.65349441e+06, 1.00000013e+00, 2.17419152e+07, 1.62723437e+09,
        8.95082567e+00, 5.24569813e+01, 1.00460494e+00, 1.44590786e+04,
        1.08951346e+01, 2.71828183e+00, 2.73223455e+02, 3.14553847e+01,
        1.58859746e+03, 1.15926939e+00, 4.94455261e+03, 7.24795681e+02,
        1.50668815e+06, 1.34684308e+03, 2.42466357e+10, 1.01468906e+02,
        2.40329671e+00, 1.54922459e+03, 1.61023463e+02, 1.47423047e+01,
        1.25981778e+02, 1.13593369e+05, 1.91867620e+00, 1.51888688e+00,
        1.46446239e+00, 1.75089213e+02, 1.30118379e+00, 1.55199001e+08,
        1.00002950e+00, 2.71812597e+00, 4.19559175e+02, 2.91805719e+01,
        1.75023828e+04, 2.19107302e+09, 5.07253660e+01, 2.33910458e+09,
        1.29413275e+04, 1.55543959e+05, 1.47938954e+09, 1.91159646e+09,
        3.72651498e+04, 1.14683026e+04, 1.77517827e+02, 1.89309192e+02,
        1.61906517e+05, 1.59573339e+07, 2.44461942e+10, 1.102277

In [65]:
pd.options.display.float_format = '{:.9f}'.format #지수함수 없이 출력하는 option

rescaled_actual_pd = pd.DataFrame(rescaled_actual)
rescaled_actual_pd

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,1.912178996,136737.136721239,18123903.122629706,1586210.462996942,1653494.412909326,1.000000132,21741915.203722715,1627234367.592443228,8.950825665,52.456981296,...,170349.430927770,13466.212357232,75.087152332,35295.804527475,1802430661.499608755,211210.659261347,148492.394305443,1.075382621,119577903103.999984741,1082121774956543.875000000


In [66]:
# len(A_config.columns)
rescaled_actual_pd.iloc[0][1]

136737.13672123852

In [67]:
for i in range (len(A_config.columns)):
    print('{} = {}'.format(A_config.columns[i], round(rescaled_actual_pd.iloc[0][i])))

automatic_sp_privileges  = 2
back_log  = 136737
binlog_cache_size  = 18123903
binlog_group_commit_sync_delay  = 1586210
binlog_group_commit_sync_no_delay_count  = 1653494
binlog_rows_query_log_events  = 1
binlog_stmt_cache_size  = 21741915
bulk_insert_buffer_size  = 1627234368
default_week_format  = 9
div_precision_increment  = 52
end_markers_in_json  = 1
eq_range_index_dive_limit  = 14459
expire_logs_days  = 11
explicit_defaults_for_timestamp  = 3
flush_time  = 273
ft_min_word_len  = 31
ft_query_expansion_limit  = 1589
general_log  = 1
group_concat_max_len  = 4945
innodb_adaptive_hash_index_parts  = 725
innodb_adaptive_max_sleep_delay  = 1506688
innodb_autoextend_increment  = 1347
innodb_buffer_pool_size  = 24246635739
innodb_change_buffer_max_size  = 101
innodb_cmp_per_index_enabled  = 2
innodb_commit_concurrency  = 1549
innodb_compression_failure_threshold_pct  = 161
innodb_compression_level  = 15
innodb_compression_pad_pct_max  = 126
innodb_concurrency_tickets  = 113593
innodb_dead